# Neo4j Utilities

## Reset Neo4j Database

Delete all nodes and relationships from the database, and remove indexes and constraints.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(f"GDS version: {gds.version()}")
assert gds.version()

### Reset Neo4j Database



In [ ]:
for _, constraint in gds.run_cypher('SHOW CONSTRAINTS yield name').iterrows():
    print(f"Removing constraint {constraint['name']}:")
    gds.run_cypher(f"DROP CONSTRAINT `{constraint['name']}`")

for _, index in gds.run_cypher('SHOW INDEXES yield name').iterrows():
    print(f"Removing index {index['name']}:")
    gds.run_cypher(f"DROP INDEX `{index['name']}`")

print("All constraints and indexes removed.")

# Delete all nodes and relationships
gds.run_cypher("MATCH (n) DETACH DELETE n")

print("All nodes and relationships deleted.")

In [ ]:
def neo4j_vector_search(question):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    CALL db.index.vector.queryNodes($index_name, $top_k, $question_embedding) 
        YIELD node, score
    RETURN score, node.text AS text
  """
  similar = []

  question_embedding = embeddings_api.embed_query(question)
  return kg.query(vector_search_query,
                      params={
                      'question': question, 
                      'question_embedding': question_embedding, 
                      'index_name':VECTOR_INDEX_NAME, 
                      'top_k': 10,
                      'openAiApiKey':OPENAI_API_KEY
                      }
                    )